## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [9]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Bredasdorp,ZA,-34.5322,20.0403,72.28,83,100,8.05,overcast clouds
1,1,Coihaique,CL,-45.5752,-72.0662,48.81,71,40,2.30,scattered clouds
2,2,Sao Gabriel Da Cachoeira,BR,-0.1303,-67.0892,74.03,94,97,1.77,overcast clouds
3,3,Caernarfon,GB,53.1413,-4.2702,45.52,70,75,24.16,broken clouds
4,4,Ushuaia,AR,-54.8000,-68.3000,44.26,70,40,8.05,scattered clouds


In [10]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [11]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
5,5,Georgetown,MY,5.4112,100.3354,89.53,64,20,4.61,few clouds
8,8,Bambous Virieux,MU,-20.3428,57.7575,84.22,89,75,12.66,light rain
10,10,Mahebourg,MU,-20.4081,57.7000,84.25,89,75,12.66,broken clouds
15,15,Atar,MR,20.5169,-13.0499,75.43,13,100,8.63,overcast clouds
25,25,Caravelas,BR,-17.7125,-39.2481,77.36,87,64,8.66,broken clouds
28,28,Busselton,AU,-33.6500,115.3333,78.08,30,97,16.64,overcast clouds
31,31,Cabedelo,BR,-6.9811,-34.8339,79.47,75,34,12.80,scattered clouds
32,32,Omboue,GA,-1.5746,9.2618,77.45,87,100,2.64,overcast clouds
37,37,Vila Velha,BR,-20.3297,-40.2925,78.75,92,20,2.30,few clouds
38,38,Harper,LR,4.3750,-7.7169,77.43,86,38,1.99,scattered clouds


In [13]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count(), preferred_cities_df.notnull().sum(), preferred_cities_df.isnull().sum()

(Unnamed: 0             124
 City                   124
 Country                124
 Lat                    124
 Lng                    124
 Max Temp               124
 Humidity               124
 Cloudiness             124
 Wind Speed             124
 Current Description    124
 dtype: int64,
 Unnamed: 0             124
 City                   124
 Country                124
 Lat                    124
 Lng                    124
 Max Temp               124
 Humidity               124
 Cloudiness             124
 Wind Speed             124
 Current Description    124
 dtype: int64,
 Unnamed: 0             0
 City                   0
 Country                0
 Lat                    0
 Lng                    0
 Max Temp               0
 Humidity               0
 Cloudiness             0
 Wind Speed             0
 Current Description    0
 dtype: int64)

In [14]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
#n/A

In [15]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
5,Georgetown,MY,89.53,few clouds,5.4112,100.3354,
8,Bambous Virieux,MU,84.22,light rain,-20.3428,57.7575,
10,Mahebourg,MU,84.25,broken clouds,-20.4081,57.7000,
15,Atar,MR,75.43,overcast clouds,20.5169,-13.0499,
25,Caravelas,BR,77.36,broken clouds,-17.7125,-39.2481,
28,Busselton,AU,78.08,overcast clouds,-33.6500,115.3333,
31,Cabedelo,BR,79.47,scattered clouds,-6.9811,-34.8339,
32,Omboue,GA,77.45,overcast clouds,-1.5746,9.2618,
37,Vila Velha,BR,78.75,few clouds,-20.3297,-40.2925,
38,Harper,LR,77.43,scattered clouds,4.3750,-7.7169,


In [18]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]
    # 6c. Get latitude and longitude from DataFrame
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
            hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError): 
        print("Hotel not found... skipping.")

hotel_df.head(10)
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
5,Georgetown,MY,89.53,few clouds,5.4112,100.3354,Cititel Penang
8,Bambous Virieux,MU,84.22,light rain,-20.3428,57.7575,Casa Tia Villa
10,Mahebourg,MU,84.25,broken clouds,-20.4081,57.7000,Shandrani Beachcomber Resort & Spa
15,Atar,MR,75.43,overcast clouds,20.5169,-13.0499,Oumou Elghoura
25,Caravelas,BR,77.36,broken clouds,-17.7125,-39.2481,Pousada dos Navegantes
28,Busselton,AU,78.08,overcast clouds,-33.6500,115.3333,Observatory Guest House
31,Cabedelo,BR,79.47,scattered clouds,-6.9811,-34.8339,Hotel Pigalle
32,Omboue,GA,77.45,overcast clouds,-1.5746,9.2618,Hotel Olako
37,Vila Velha,BR,78.75,few clouds,-20.3297,-40.2925,Quality Suites Vila Velha
38,Harper,LR,77.43,scattered clouds,4.3750,-7.7169,Screensaver


In [20]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.count(), hotel_df.notnull().sum(), hotel_df.isnull().sum()

(City                   124
 Country                124
 Max Temp               124
 Current Description    124
 Lat                    124
 Lng                    124
 Hotel Name             124
 dtype: int64,
 City                   124
 Country                124
 Max Temp               124
 Current Description    124
 Lat                    124
 Lng                    124
 Hotel Name             124
 dtype: int64,
 City                   0
 Country                0
 Max Temp               0
 Current Description    0
 Lat                    0
 Lng                    0
 Hotel Name             0
 dtype: int64)

In [23]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [26]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City Name</dt><dd>{City}</dd>
<dt>Country Code</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd><dd>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [27]:
# 11a. Add a marker layer for each city to the map. 
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))